In [0]:
%sql
use catalog ecommerce;
use schema  v01;

# A. Create source and target tables 

In [0]:
%sql
create or replace table users as
select * from
 read_files(
  "/Volumes/workspace/default/adventure_works/Users.csv",
  format => "csv"
 );
 select * from users

In [0]:
%sql
create or replace table update_users_source as
select * from
 read_files(
  "/Volumes/workspace/default/adventure_works/Users-updated.csv",
  format => "csv"
 );

 select * from update_users_source;

In [0]:
%sql
create or replace table new_users as
select * from
 read_files(
  "/Volumes/workspace/default/adventure_works/Users-new.csv",
  format => "csv"
 );

 select * from new_users;

# B. Mege Into

In [0]:
%sql
MERGE INTO ecommerce.v01.users AS target
USING ecommerce.v01.update_users_source AS source 
ON source.CustomerKey= target.CustomerKey
WHEN MATCHED AND source.Status="update" THEN
UPDATE SET target.Prefix = source.Prefix, 
target.LastNamener = source.LastNamener,
target.FirstName = source.FirstName,
target.status = source.Status
WHEN MATCHED AND source.Status="delete" THEN 
DELETE 
WHEN NOT MATCHED THEN 
INSERT * ;

SELECT * FROM ecommerce.v01.users


In [0]:
%sql
DESCRIBE HISTORY ecommerce.v01.users 

In [0]:
%sql
SELECT * 
FROM ecommerce.v01.users VERSION AS OF 2;

# C. Schema Enforcement and Schema Evolution with MERGE INTO

In [0]:
%sql
MERGE WITH SCHEMA EVOLUTION INTO ecommerce.v01.users AS target
USING ecommerce.v01.new_users AS source
ON source.CustomerKey= target.CustomerKey
WHEN MATCHED AND source.Status= "update" THEN
UPDATE SET target.Prefix = source.Prefix, 
target.LastNamener = source.LastNamener,
target.FirstName = source.FirstName,
target.status = source.Status
WHEN MATCHED AND source.Status = "delete" THEN 
DELETE 
WHEN NOT MATCHED THEN 
INSERT * ;

SELECT * FROM ecommerce.v01.users;